# Mayan Biosphere Reserve

The Maya Biosphere Reserve (MBR) is the largest protected ecosystem in Mesoamerica, composed of about 14 percent of the surface of Guatemala (Maya Biosphere Reserve Living Landscape, USAID). The MBR is composed of protected areas, multi-use zones, and buffer zones at the southern edge of the reserve.

In the late 1990s, the Guatemalan government granted concessions to local communities for forest use in multi-use zones. These concessions lasted for 25 years, with concessions being renewed on an ongoing basis by the federal government. Around 36% of the MBR’s area prohibits human settlement, logging, or extraction. Established multi-use zones 36% as well as a where silviculture and agroforestry are permitted (Protected Planet, 2024,
Preferred by Nature, 2024). The multi-use zone makes up around 40% of the reserve boundaries and allow communities to utilize silviculture and agroforestry (Protected Planet, 2024, Preferred by Nature, 2024). Lastly, a buSer zone comprising of around 24% of the reserve area permits certain regulated economic activities, such as timber and non-timber forest product extraction (Protected Planet, 2024, Preferred by Nature, 2024).


In [4]:
import ee
import geemap
import geopandas as gpd

In [5]:
ee.Authenticate()
ee.Initialize()

In [6]:
project_gdf = gpd.read_file(
    "../../geometries/WDPA_WDOECM_Nov2024_Public_26621_shp/WDPA_WDOECM_Nov2024_Public_26621_shp-polygons.shp"
)
project_fc = ee.FeatureCollection(project_gdf.__geo_interface__)

# Forest Loss/Gains

In the past 20 years (2002-2022), the primary LULC category is the transition from forest to cropland and grassland. The transition to grassland can be observed particularly in the Parque Nacional Laguna del Tigre in the northwest of the MBR. This area is part of a National Park where logging, settlements, and human disturbances are prohibited.

The transition from forest to cropland can be seen in the MBR’s southern buffer zone, which allow the sale of land by federal authorities to individuals for the purpose of sustainable farming. Because the MBR is located at the border of Guatemala and Mexico, it receives migrant populations that have been known to burn land and illegally use forest resources (Carroso et al., 2000).

In [7]:
hansen = ee.Image("UMD/hansen/global_forest_change_2023_v1_11")
hansen = hansen.clip(project_fc)

In [17]:
# Create an animation of forest loss by year

lossyear = hansen.select("lossyear")
canopy_cover_2000 = hansen.select("treecover2000")

In [9]:
lossyear

In [27]:
loss_ramp = {
    "palette": "cividis",
    "min": 0,
    "max": 23,
}

In [29]:
p = geemap.Map()
p.addLayer(lossyear, loss_ramp, "Year of Forest Loss")
p.centerObject(project_fc)
p.add_text("Made by Ilse Paniagua")
p.add_colorbar(
    loss_ramp,
    label="Year of Forest Loss (2000-2023)",
    layer_name="Year of Forest Loss",
    orientation="horizontal",
)
p

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

![Year of Forest Loss](../../maps/mayan_biosphere_year_of_forest_loss.png)

## Copernicus Global Land Cover Layers: CGLS-LC100 

IPCC has the following Land Use / Land Cover (LULC) definitions:
Forest land, Cropland, Grassland, Wetlands, Settlements, and Other land

To select a LCLU product, the following considerations were used:
* Data availability for at least 2004-2023, on a yearly basis
* As close to IPCC categories as possible, with the possibility of collapsing more granular categories into more general groupings

Products explored:
* The European Space Agency (ESA) WorldCover product had categories that were closest to the IPCC definition, but only 2020 and 2021
* GlobCover: Global Land Cover Map was only available for 2009-2010
* Copernicus Global Land Cover Layers are available from 2015-2019 and have more granular categories than IPCC
* Dynamic World V1 has data from 2015-2024 with categories that align with IPCC
* IPCC Land Use Classification - CCI/ESA. Available 1992-2022. Categories align with IPCC, but does not appear to be available in Google Earth Engine.

For this analysis, we use Copernicus Global Land Cover Layers and translate between classes to the IPCC Land Use Category.

| CCI-LC Class | Description | IPCC Land Use Category |  |
|---|---|---|---|
| 10, 11, 12 | Rainfed cropland | Cropland |  |
| 20 | Irrigated cropland | Cropland |  |
| 30 | Mosaic cropland (>50%) / natural vegetation (<50%) | Cropland |  |
| 40 | Mosaic natural vegetation (>50%) / cropland (<50%) | Cropland |  |
| 50 | Tree cover, broadleaved, evergreen, closed to open (>15%) | Forest |  |
| 60, 61, 62 | Tree cover, broadleaved, deciduous, closed to open (>15%) | Forest |  |
| 70, 71, 72 | Tree cover, needleleaved, evergreen, closed to open (>15%) | Forest |  |
| 80, 81, 82 | Tree cover, needleleaved, deciduous, closed to open (>15%) | Forest |  |
| 90 | Tree cover, mixed leaf type (broadleaved and needleleaved) | Forest |  |
| 100 | Mosaic tree and shrub (>50%) / herbaceous cover (<50%) | Forest |  |
| 110 | Mosaic herbaceous cover (>50%) / tree and shrub (<50%) | Grassland |  |
| 120, 121, 122 | Shrubland | Grassland |  |
| 130 | Grassland | Grassland |  |
| 140 | Lichens and mosses | Grassland |  |
| 150, 151, 152, 153 | Sparse vegetation (tree, shrub, herbaceous cover) | Grassland |  |
| 160 | Tree cover, flooded, fresh or brakish water | Wetland |  |
| 170 | Tree cover, flooded, saline water | Wetland |  |
| 200, 201, 202 | Bare areas | Otherland |  |
| 210 | Water Bodies | No data |  |
| 220 | Permanent snow and ice | Otherland |  |

In [ ]:
land_cover_collection = ee.Image(
    "COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019"
).select("discrete_classification")
land_cover_collection

## Biomass Estimation

* Global Aboveground and Belowground Biomass Carbon Density Maps from NASA ORNL DAAC at Oak Ridge National Laboratory was produced for 2010 (Aboveground living biomass carbon stock and belowground carbon stock as separate bands) at 300 meter resolution. This product also provides an uncertainty band for both AGB and BGB.
*  WCMC Above and Below Ground Biomass Carbon Density is available in tropical countries, also produced in the year 2010 (Tonnes of above and below ground biomass carbon per hectare as a single band) at 300 meter resolution
*  WHRC Pantropical National Level Carbon Stock Dataset was produced for the year 2012 (Megagrams of aboveground live woody biomass per Hectare) at 500m resolution
  
The  Global Aboveground and Belowground Biomass Carbon Density product was chosen for this analysis because it provides uncertainty bands and estimates for AGB and BGB as separate bands.

In [11]:
biomass = ee.ImageCollection("NASA/ORNL/biomass_carbon_density/v1")

In [33]:
agb = biomass.select("agb").median()

In [34]:
agb = agb.clip(project_fc)

In [40]:
agb_viz = {"palette": "viridis", "min": 0, "max": 129}

In [43]:
w = geemap.Map()
w.addLayer(agb, agb_viz, "Above Ground Biomass (Oak Ridge Lab 2010)")
w.centerObject(project_fc)
w.add_text("Made by Ilse Paniagua")
w.add_colorbar(
    agb_viz,
    label="Above Ground Biomass",
    layer_name="Above Ground Biomass (Oak Ridge Lab 2010",
    orientation="horizontal",
)
w

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

![](../../maps/mayan_biosphere_AGB.png)